# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

In [25]:
#Importing important modules
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
#Installing Tensorboard
!pip install Tensorboard

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#### Import the mnist dataset from keras datasets

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [44]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [45]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
x_train_lt5 = x_train_lt5.reshape(x_train_lt5.shape[0],28,28,1)

x_test_lt5 = x_test_lt5.reshape(x_test_lt5.shape[0],28,28,1)


### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
x_train = x_train_lt5.astype('float32')
x_test = x_test_lt5.astype('float32')

#Normalizing the input
x_train /= 255.0
x_test /= 255.0


### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [48]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print('X_test shape:', x_test.shape)
print(x_test.shape[0], 'test samples')

x_train shape: (30596, 28, 28, 1)
30596 train samples
X_test shape: (5139, 28, 28, 1)
5139 test samples


In [49]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:

num_classes = 5


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_lt5, num_classes)
y_test = keras.utils.to_categorical(y_test_lt5, num_classes)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [0]:
#Initialize the model
model = Sequential()

#Add a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'))

#Add a Convolutional Layer with 64 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(64, (3, 3), activation='relu'))

#Add a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2)))

#Apply Dropout with 0.25 probability 
model.add(Dropout(0.25))


### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:
#Flatten the layer
model.add(Flatten())

#Add Fully Connected Layer with 128 units and activation function as 'ReLU'
model.add(Dense(128, activation='relu'))

#Apply Dropout with 0.5 probability 
model.add(Dropout(0.5))

#Add Fully Connected Layer with 10 units and activation function as 'softmax'
model.add(Dense(num_classes, activation='softmax'))

In [0]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)
#Set the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

### Print the training and test accuracy for 5 epochs

In [55]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,validation_data=(x_test, y_test))

Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 5s 168us/step - loss: 0.1247 - acc: 0.9608 - val_loss: 0.0170 - val_acc: 0.9955
Epoch 2/5
30596/30596 [==============================] - 5s 149us/step - loss: 0.0333 - acc: 0.9899 - val_loss: 0.0108 - val_acc: 0.9959
Epoch 3/5
30596/30596 [==============================] - 5s 148us/step - loss: 0.0241 - acc: 0.9931 - val_loss: 0.0074 - val_acc: 0.9975
Epoch 4/5
30596/30596 [==============================] - 5s 149us/step - loss: 0.0169 - acc: 0.9948 - val_loss: 0.0065 - val_acc: 0.9975
Epoch 5/5
30596/30596 [==============================] - 5s 149us/step - loss: 0.0135 - acc: 0.9955 - val_loss: 0.0040 - val_acc: 0.9986


In [56]:
#Testing the model on test set
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5139/5139 [==============================] - 0s 96us/step
Test loss: 0.004002713897759821
Test accuracy: 0.9986378672893559


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [57]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_3
conv2d_3is not trainable

conv2d_4
conv2d_4is not trainable

max_pooling2d_2
max_pooling2d_2is not trainable

dropout_3
dropout_3is not trainable

flatten_2
flatten_2is not trainable

dense_3
dense_3 is trainable

dropout_4
dropout_4is not trainable

dense_4
dense_4 is trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [0]:
#Reshape
x_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0],28,28,1)

x_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0],28,28,1)

#Datatype change
x_train_tl = x_train_gt5.astype('float32')
x_test_tl = x_test_gt5.astype('float32')

#Normalizing the input
x_train_tl /= 255.0
x_test_tl /= 255.0
# convert class vectors to binary class matrices
y_train_tl = keras.utils.to_categorical(y_train_gt5, num_classes)
y_test_tl = keras.utils.to_categorical(y_test_gt5, num_classes)

### Check

Verify the shapes with the given below.

In [68]:
print(x_train_tl.shape)
print(y_train_tl.shape)
print(x_test_tl.shape)
print(y_test_tl.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


In [59]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404,)
(4861, 28, 28, 1)
(4861,)


## Print the accuracy for classification of digits 5 to 9

In [0]:
#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)
#Set the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

In [72]:
model.fit(x_train_tl, y_train_tl,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(x_test_tl, y_test_tl))

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 3s 104us/step - loss: 0.3768 - acc: 0.9043 - val_loss: 0.0502 - val_acc: 0.9844
Epoch 2/5
29404/29404 [==============================] - 2s 84us/step - loss: 0.0846 - acc: 0.9747 - val_loss: 0.0341 - val_acc: 0.9887
Epoch 3/5
29404/29404 [==============================] - 2s 83us/step - loss: 0.0584 - acc: 0.9823 - val_loss: 0.0303 - val_acc: 0.9893
Epoch 4/5
29404/29404 [==============================] - 2s 82us/step - loss: 0.0468 - acc: 0.9859 - val_loss: 0.0243 - val_acc: 0.9928
Epoch 5/5
29404/29404 [==============================] - 2s 82us/step - loss: 0.0392 - acc: 0.9876 - val_loss: 0.0282 - val_acc: 0.9909


In [73]:
#Testing the model on test set
score = model.evaluate(x_test_tl, y_test_tl)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

4861/4861 [==============================] - 1s 107us/step
Test loss: 0.02816057210300251
Test accuracy: 0.9909483645340464


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [7]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [8]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [9]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
X_train = twenty_train.data
X_test = twenty_test.data
y_train = twenty_train.target
y_test = twenty_test.target

In [13]:

# use TfidfVectorizer with text column only
vect = TfidfVectorizer(stop_words= 'english', ngram_range=(1, 2), min_df=2, max_df= 0.9)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
print(X_train_dtm.shape)
print(X_test_dtm.shape)

(2257, 65240)
(1502, 65240)


### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [14]:
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_test_dtm)
y_train_pred_class = logreg.predict(X_train_dtm)
print('Accuracy for Test(LG):',metrics.accuracy_score(y_test, y_pred_class))
print('Accuracy for Train(LG):', metrics.accuracy_score(y_train,y_train_pred_class))

Accuracy for Test(LG): 0.9307589880159787
Accuracy for Train(LG): 1.0
